In [ ]:
from numba import njit, prange
import numpy as np
import xarray as xr
import random
import time
import matplotlib.pyplot as plt
from math import sin

h = 6.62607004e-34
e = 1.6021765e-19
hbar = h / (2 * np.pi)

# Shapiro maps

In [15]:

def RSJ_SQUID(I_dc: np.array, 
           I_rf: np.array,
           f_rf: float, 
           Rj: float, 
           Ic: float, 
           alpha :float = 0.,
           beta :float = 1.,
           gamma :float = 0.,
           rho :float = 0,
           phi_ext :float = 0.5,
           sigma: float = 0.5, 
           n_periods: int = 100, 
           pts_period: int = 1000,
           simulation_type = 'I-V') -> xr.DataArray:

    t0 = hbar / ( 2 * e * Ic * Rj )
    eta = 2 * np.pi * f_rf * t0

    # Time step of the t vs phi evolution
    dt = 2 * np.pi / ( eta * pts_period )
    n_total = n_periods * pts_period

    params = {'f_rf': f_rf, 'Rj': Rj, 'Ic': Ic, 'sigma': sigma, 'n_periods': n_periods, 'pts_period': pts_period}
    
    @njit
    def CPR(phi):
        """ Return CPR of the junction """
        
        return sin(phi)

    @njit
    def Dphi(phi, t, i_dc, i_rf):
        """ Return derivative of phases """
        j = (phi[0] - phi[1] - 2*np.pi*phi_ext)/(np.pi*beta) - 0.5 * gamma * i_dc

        dphi1 = (0.5 * i_dc + 0.5 * i_rf * sin(eta*t) - j - ( 1 - alpha ) * CPR(phi[0]))/(1 - rho)
        dphi2 = (0.5 * i_dc + 0.5 * i_rf * sin(eta*t) + j - ( 1 + alpha ) * CPR(phi[1]))/(1 + rho)
        return np.array([dphi1, dphi2])

    @njit
    def RK2(phi, t, i_dc, i_rf):
        """ 4th order Runge-Kutta method """

        F1 = Dphi(phi, t, i_dc, i_rf)
        F2 = Dphi(phi + 0.5 * dt * F1, t + 0.5 * dt, i_dc, i_rf)
        F3 = Dphi(phi + 0.5 * dt * F2, t + 0.5 * dt, i_dc, i_rf)
        F4 = Dphi(phi + dt * F3, t + dt, i_dc, i_rf)

        phi_out = phi + dt/6. * (F1 + F4 + 2.*(F2 + F3))
        t_out = t + dt

        return phi_out, t_out

    @njit
    def calculate(i_dc, i_rf, phi0):
        phi, t = np.zeros((n_total,2)), np.zeros(n_total)
        phi[0], t[0] = phi0, 0

        for n in range(n_total - 1):
            noise = random.gauss(0,sigma)
            phi[n+1], t[n+1] = RK2(phi[n], t[n], i_dc + noise, i_rf)

        return (phi[-1, 0] - phi[0, 0])/(t[-1]-t[0]), np.array([phi[-1, 0], phi[-1, 1]])

    @njit(parallel=True)
    def loop(V, I_dc, I_rf):

        for n1 in prange(len(I_rf)):
            V[0, n1], phi0 = calculate(I_dc[0], I_rf[n1], phi0 = np.array([0,0]))

            for n2 in range(len(I_dc)):
                V[n2, n1], phi0  = calculate(I_dc[n2], I_rf[n1], phi0 = phi0)

        return V * Ic * Rj / ( h * f_rf / ( 2 * e ) )

    """ Run the simulation """
    start = time.time()

    V = np.zeros((len(I_dc), len(I_rf)))
    V = xr.DataArray(loop(V, I_dc, I_rf), 
                    coords = {'i_dc': I_dc, 'i_rf': I_rf}, 
                    attrs = {'long_name': 'Voltage drop', 
                              'units': '$hf/2e$',
                              'params': params})


    elapsed_time = time.time() - start
    print(f'Simulation took: {elapsed_time:.1f} s')
    
    return V

In [ ]:
I_dc = np.linspace(-2,2, 100)
I_rf = np.logspace(-0.5,0.5, 50)
V = RSJ_SQUID(I_dc, I_rf, f_rf = 1.75e9, Rj = 500, Ic = 50e-9, sigma = 0., alpha=0.15, beta=10, gamma=0, rho=0, phi_ext=0.)

V.transpose().differentiate('i_dc').plot()

# Flux sweeps

In [16]:
def RSJ_SQUID(I_dc: np.array, 
           Phi_ext: np.array,
           i_rf: float,
           f_rf: float, 
           Rj: float, 
           Ic: float, 
           alpha :float = 0.,
           beta :float = 1.,
           gamma :float = 0.,
           rho :float = 0,
           sigma: float = 0.5, 
           n_periods: int = 100, 
           pts_period: int = 1000) -> xr.DataArray:

    t0 = hbar / ( 2 * e * Ic * Rj )
    eta = 2 * np.pi * f_rf * t0

    # Time step of the t vs phi evolution
    dt = 2 * np.pi / ( eta * pts_period )
    n_total = n_periods * pts_period

    params = {'f_rf': f_rf, 'Rj': Rj, 'Ic': Ic, 'sigma': sigma, 'n_periods': n_periods, 'pts_period': pts_period}
    
    @njit
    def CPR(phi):
        """ Return CPR of the junction """
        
        return sin(phi)

    @njit
    def Dphi(phi, t, i_dc, phi_ext):
        """ Return derivative of phases """
        j = (phi[0] - phi[1] - 2*np.pi*phi_ext)/(np.pi*beta) - 0.5 * gamma * i_dc

        dphi1 = (0.5 * i_dc + 0.5 * i_rf * sin(eta*t) - j - ( 1 - alpha ) * CPR(phi[0]))/(1 - rho)
        dphi2 = (0.5 * i_dc + 0.5 * i_rf * sin(eta*t) + j - ( 1 + alpha ) * CPR(phi[1]))/(1 + rho)
        return np.array([dphi1, dphi2])

    @njit
    def RK2(phi, t, i_dc, phi_ext):
        """ 2th order Runge-Kutta method """

        F1 = Dphi(phi, t, i_dc, phi_ext)
        F2 = Dphi(phi + 0.5 * dt * F1, t + 0.5 * dt, i_dc, phi_ext)
        F3 = Dphi(phi + 0.5 * dt * F2, t + 0.5 * dt, i_dc, phi_ext)
        F4 = Dphi(phi + dt * F3, t + dt, i_dc, phi_ext)

        phi_out = phi + dt/6. * (F1 + F4 + 2.*(F2 + F3))
        t_out = t + dt

        return phi_out, t_out

    @njit
    def calculate(i_dc, phi_ext, phi0):
        phi, t = np.zeros((n_total,2)), np.zeros(n_total)
        phi[0], t[0] = phi0, 0

        for n in range(n_total - 1):
            noise = random.gauss(0,sigma)
            phi[n+1], t[n+1] = RK2(phi[n], t[n], i_dc + noise, phi_ext)

        return (phi[-1, 0] - phi[0, 0])/(t[-1]-t[0]), np.array([phi[-1, 0], phi[-1, 1]])

    @njit(parallel=True)
    def loop(V, I_dc, Phi_ext):

        for n1 in prange(len(Phi_ext)):
            V[0, n1], phi0 = calculate(I_dc[0], Phi_ext[n1], phi0 = np.array([0,0]))

            for n2 in range(len(I_dc)):
                V[n2, n1], phi0  = calculate(I_dc[n2], Phi_ext[n1], phi0 = phi0)

        return V * Ic * Rj


    """ Run the simulation """
    start = time.time()

    V = np.zeros((len(I_dc), len(Phi_ext)))
    V = xr.DataArray(loop(V, I_dc, Phi_ext), 
                    coords = {'i_dc': I_dc, 'phi_ext': Phi_ext}, 
                    attrs = {'long_name': 'Voltage drop', 
                              'units': 'V',
                              'params': params})


    elapsed_time = time.time() - start
    
    print(f'Simulation took: {elapsed_time:.1f} s')
    
    return V

In [ ]:
I_dc = np.linspace(-3,3, 100)
Phi_ext = np.linspace(-0.75,0.75,50)

V = RSJ_SQUID(I_dc, Phi_ext, i_rf=0, f_rf = 1.75e9, Rj = 500, Ic = 50e-9, sigma = 0., alpha=0, beta=1, gamma=0, rho=0, )

In [ ]:
V.differentiate('i_dc').plot()